In [1]:
import os
import sys

ROOT = os.getcwd()
WORK = os.path.join(ROOT, "work")
MODEL = "ssd_mobilenet_v1_coco_2018_01_28"
os.makedirs(WORK, exist_ok=True)

# force tf2onnx to cpu
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['MODEL'] = MODEL
os.environ['WORK'] = WORK

In [2]:
!pip install --user -U tf2onnx
!pip install --user -U onnxruntime

Requirement already up-to-date: tf2onnx in /home/ubuntu/.local/lib/python3.7/site-packages (1.9.3)
Requirement already up-to-date: onnxruntime in /home/ubuntu/.local/lib/python3.7/site-packages (1.10.0)


In [12]:
!cd $WORK; wget -q http://download.tensorflow.org/models/object_detection/$MODEL.tar.gz
!cd $WORK; tar zxvf $MODEL.tar.gz

ssd_mobilenet_v1_coco_2018_01_28/
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.index
ssd_mobilenet_v1_coco_2018_01_28/checkpoint
ssd_mobilenet_v1_coco_2018_01_28/pipeline.config
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.data-00000-of-00001
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.meta
ssd_mobilenet_v1_coco_2018_01_28/saved_model/
ssd_mobilenet_v1_coco_2018_01_28/saved_model/saved_model.pb
ssd_mobilenet_v1_coco_2018_01_28/saved_model/variables/
ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb


In [2]:
!saved_model_cli show --dir $WORK/$MODEL/saved_model/ --tag_set serve  --signature_def serving_default

2022-01-25 15:51:42.568466: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['inputs'] tensor_info:
      dtype: DT_UINT8
      shape: (-1, -1, -1, 3)
      name: image_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100, 4)
      name: detection_boxes:0
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_classes:0
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_scores:0
  outputs['num_detections'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: num_detections:0
Method name is: tensorflow/serving/predict


In [4]:
!saved_model_cli show --dir weights/saved_model/ --tag_set serve  --signature_def serving_default

2022-01-25 15:52:06.636902: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['input_tensor'] tensor_info:
      dtype: DT_UINT8
      shape: (1, -1, -1, 3)
      name: serving_default_input_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_anchor_indices'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:0
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 4)
      name: StatefulPartitionedCall:1
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:2
  outputs['detection_multiclass_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 10)
      name: StatefulPartitionedCall:3
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLO

In [7]:
!python -m tf2onnx.convert --opset 13 --saved-model weights/saved_model --output saved_model.onnx

2022-01-25 15:53:01.178988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/anaconda3/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-01-25 15:53:02.470777: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-25 15:53:02.471680: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-25 15:53:02.514167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-25 15:53:02.514748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 

2022-01-25 15:53:12.679237: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 3453 nodes (3119), 6817 edges (6476), time = 93.319ms.
  function_optimizer: Graph size after: 3453 nodes (0), 6817 edges (0), time = 39.98ms.
Optimization results for grappler item: __inference_map_while_cond_7224_28043
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.
Optimization results for grappler item: __inference_map_while_body_7225_11561
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2022-01-25 15:53:15.547625: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-25 15:53:15.547828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA n

2022-01-25 15:53:16.590569: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 1441 nodes (-1753), 4616 edges (-1930), time = 213.936ms.
  function_optimizer: Graph size after: 1441 nodes (0), 4616 edges (0), time = 34.324ms.
  constant_folding: Graph size after: 1441 nodes (0), 4616 edges (0), time = 45.582ms.
  function_optimizer: Graph size after: 1441 nodes (0), 4616 edges (0), time = 31.376ms.
Optimization results for grappler item: __inference_map_while_cond_7224_28043
  constant_folding: Graph size after: 12 nodes (0), 8 edges (0), time = 0.36ms.
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  constant_folding: Graph size after: 12 nodes (0), 8 edges (0), time = 0.214ms.
  function_optimizer: function_optimizer did nothing. time = 0.006ms.
Optimization results for grappler item: __inference_map_while_body_7225_11561
  constant_folding: Graph size afte

2022-01-25 15:53:18.644888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-25 15:53:18.645287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-01-25 15:53:18.645330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-25 15:53:18.645359: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-25 15:53:18.645379: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-25 15:53:18.645398: I tensorflow/stream_executor/platform/defa

2022-01-25 15:53:19,576 - INFO - folding node using tf type=Select, name=StatefulPartitionedCall/Postprocessor/BatchMultiClassNonMaxSuppression/PadOrClipBoxList/Select_1
2022-01-25 15:53:19,576 - INFO - folding node using tf type=Select, name=StatefulPartitionedCall/Postprocessor/BatchMultiClassNonMaxSuppression/PadOrClipBoxList/Select_8
2022-01-25 15:53:20,536 - INFO - Optimizing ONNX model
2022-01-25 15:53:27,504 - INFO - After optimization: BatchNormalization -53 (60->7), Cast -108 (221->113), Const -930 (1179->249), Identity -64 (64->0), Less -1 (13->12), Mul -2 (57->55), Placeholder -2 (6->4), ReduceSum -9 (10->1), Reshape -41 (80->39), Shape -10 (33->23), Slice -9 (64->55), Squeeze -11 (65->54), Sub -9 (40->31), Transpose -270 (291->21), Unsqueeze -20 (66->46)
2022-01-25 15:53:27,586 - INFO - 
2022-01-25 15:53:27,586 - INFO - Successfully converted TensorFlow model DB-AI_ONNX/weights7/saved_model to ONNX
2022-01-25 15:53:27,586 - INFO - Model inputs: ['input_tensor']
2022-01-25 1